In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot as plt
from pandas import Series
from sklearn.metrics import mean_squared_error
from datetime import datetime 

In [ ]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [ ]:
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [ ]:
INPUT_PATH = '../../../data/processed'
INPUT_FILE_NAME = 'dataproc_v001'
OUTPUT_PATH = '../../../models/arima/hyperparameters/'
HYPERPARAM_NAME = 'best_hyperparam_sarima_r'
LOG_NAME = 'rsearch_arima_logs_r'
OUTPUT_FILE_NAME = 'rsearch_sarima_logs_d'
NITER = 200
NRUN = 3
DAYS_PRED = 28
METRIC = 'rmse'

### FUNCTIONS

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, 
                    trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

In [ ]:
def walk_forward_validation(train, test, cfg, metric='rmse'):
    
    error = 0
    
    n_test = test.shape[0]
    
    for test in test:
  
        history = [x for x in train]
        # make predictions
        predictions = list()
        for t in test:           
            # fit model and make forecast for history
            yhat = sarima_forecast(history, cfg)
            predictions.append(yhat)
            history.append(t)

        # calculate out of sample error
        if metric=='mse':
            error += mean_squared_error(test, predictions)
        elif metric == 'rmse':
            error += rmse(test, predictions)



    return error/n_test

In [ ]:
# score a model, return None on failure
def score_model(train, test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(train, test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                tic = datetime.now()
                result = walk_forward_validation(train, test, cfg)
                toc = datetime.now()
        except:
            result = None
    # check for an interesting result
    if result is not None:
        #print(' > Model[%s] %.3f' % (key, result))
        diff_time_run = toc - tic
        line = datetime.now().strftime("%d/%m/%Y") + ", " + str(key) + ", " + METRIC + ", " + str(result) + ", " + str(diff_time_run.total_seconds()/60) + "\n"
        
        # save into log file
        with open(f'{OUTPUT_PATH}/{LOG_NAME}{NRUN}.csv','a+') as f:
            f.write(line)
        
    return (key, result)

In [ ]:
# grid search configs
def grid_search(train, test, cfg_list, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(train, test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(train, test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

In [ ]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [i for i in range(11)]
	d_params = [i for i in range(4)]
	q_params = [i for i in range(4)]
	t_params = ['n','c','t','ct']
	P_params = [i for i in range(4)]
	D_params = [0, 1, 2]
	Q_params = [i for i in range(4)]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

### LOAD DATASET

In [ ]:
data = pd.read_pickle(f'{INPUT_PATH}/{INPUT_FILE_NAME}.pkl')

In [ ]:
data = data[data.part == 'train'] # select only train data

In [ ]:
date_cutoff = data.d.max() - DAYS_PRED

X_train = data[data.d <= date_cutoff]

X_test = data[data.d > date_cutoff]

del data

In [ ]:
LEVEL = ['state_id', 'd']

In [ ]:
train_agg = X_train.groupby(LEVEL).demand.mean().reset_index()

In [ ]:
with open(f'{OUTPUT_PATH}/{LOG_NAME}{NRUN}.csv','w+') as f:
    f.write(f"date, params, metric, score, time [min] \n")

### TRAIN MODEL

In [ ]:
# model configs
cfg_list = sarima_configs()

In [ ]:
# select random parameters 
ncfg = len(cfg_list)
rcfg = np.random.choice(ncfg, size=NITER, replace=False)

In [ ]:
random_cfg_list =  np.array(cfg_list)[rcfg,:].tolist()

In [ ]:
ID = 2
train = train_agg[train_agg.state_id == ID].demand.tolist()

In [ ]:
test = X_test[X_test.state_id == ID].pivot(index='id', columns='d', values='demand').values

In [ ]:
nrows = test.shape[0]

In [ ]:
ridx = np.random.choice(nrows, size=100, replace=False)

In [ ]:
test = test[ridx, :] # select only some rows to test because take to many time to evaluate the model

In [ ]:
# grid search
scores = grid_search(train, test, random_cfg_list)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)